In [1]:
def racha_detection(lista):
    # It returns a list of lists where each sub-list contains the consecutive tokens in the list
    rachas = []
    racha = []
    for i, element in enumerate(lista):
        if (i<len(lista)-1) and (lista[i+1] == element+1):
            racha.append(element)
        else:
            if len(racha)>0:
                rachas.append(racha + [element])          
            else:# (i!=len(lista)-1):
                rachas.append([element])
            racha = []
    return rachas

def masking(tokenized_sentence, rachas):
    # Function to mask a tokenized_sentence (token ids) following the rachas described in rachas
    # Only one sentinel_token per racha
    sent_token_id = 0
    enmascared = tokenized_sentence.copy()
    for racha in rachas:
        sent_token = f'<extra_id_{sent_token_id}>'
        sent_id = tokenizer.encode(sent_token)[0]
        for i, idx in enumerate(racha):
            if i==0:
                enmascared[idx] = sent_id
            else:
                enmascared[idx] = -100
        sent_token_id += 1
        
    enmascared = [t for t in enmascared if t!=-100] 

    return enmascared

def add_noise(sentence, tokenizer, percent=0.15):
    # Function that takes a sentence, tokenizer and a noise percentage and returns
    # the masked input_ids and masked target_ids accordling with the T5 paper and HuggingFace docs
    # To see the process working uncomment all the prints ;)
    tokenized_sentence = tokenizer.encode(sentence)
    #print('PRE-MASKED:')
    #print('INPUT: {}'.format(tokenizer.convert_ids_to_tokens(tokenized_sentence)))
   
    idxs_2_mask = sorted(random.sample(range(len(tokenized_sentence)), 
                                       int(len(tokenized_sentence)*percent)))
    rachas = racha_detection(idxs_2_mask)
    enmascared_input = masking(tokenized_sentence, rachas)
    #print('RACHAS INPUT: {}'.format(rachas))
    idxs_2_mask = [idx for idx in range(len(tokenized_sentence)) if idx not in idxs_2_mask]
    rachas = racha_detection(idxs_2_mask)
    enmascared_target = masking(tokenized_sentence, rachas)
    #print('RACHAS TARGET: {}'.format(rachas))
    
    #print('POST-MASKED:')
    #print('INPUT: {}'.format(tokenizer.convert_ids_to_tokens(enmascared_input)))
    #print('TARGET: {}'.format(tokenizer.convert_ids_to_tokens(enmascared_target)))

    return enmascared_input, enmascared_target

In [2]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration

T5_PATH = 'sonoisa/t5-base-japanese-v1.1'
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)

In [3]:
import random

In [4]:
tokenizer = t5_tokenizer

In [8]:
text = 'Toxic shock syndrome(TSS)は黄色ぶどう球菌やA群 β 溶血性連鎖球菌が産生するスーパー抗原に起因した感染性疾患である.'

outputs = add_noise(text, tokenizer = tokenizer)

In [13]:
print('INPUT: {}'.format(tokenizer.convert_ids_to_tokens(outputs[0])))

INPUT: ['▁', 'T', 'ox', 'ic', '▁', '<extra_id_0>', 'syn', 'dr', 'ome', '(', 'T', '<0x53>', '<extra_id_1>', ')', 'は', '黄色', '<extra_id_2>', 'どう', '球', '菌', 'や', '<extra_id_3>', '群', '▁', 'β', '▁', '溶', '血', '性', '連鎖', '球', '菌', 'が', '産', '生', 'する', 'スーパー', '抗', '原', 'に', '起', '因', 'した', '<extra_id_4>', '性', '疾患', 'である', '.', '</s>']


In [14]:
print('TARGET: {}'.format(tokenizer.convert_ids_to_tokens(outputs[1])))

TARGET: ['<extra_id_0>', 'sho', 'ck', '▁', '<extra_id_1>', '<0x53>', '<extra_id_2>', 'ぶ', '<extra_id_3>', '<0x41>', '<extra_id_4>', '感染', '<extra_id_5>']


In [5]:
text = 'Toxicshocksyndrome(TSS)は黄色ぶどう球菌やA群β溶血性連鎖球菌が産生するスーパー抗原に起因した感染性疾患である.'

outputs = add_noise(text, tokenizer = tokenizer)

In [6]:
print('INPUT: {}'.format(tokenizer.convert_ids_to_tokens(outputs[0])))

INPUT: ['▁', '<extra_id_0>', 'ox', 'ic', 'sho', 'ck', 'syn', 'dr', 'ome', '(', 'T', '<0x53>', '<0x53>', ')', 'は', '黄色', 'ぶ', 'どう', '球', '菌', 'や', '<0x41>', '群', 'β', '溶', '血', '<extra_id_1>', '菌', '<extra_id_2>', '生', 'する', '<extra_id_3>', '抗', '原', 'に', '起', '因', 'した', '感染', '性', '疾患', 'である', '.', '</s>']
